# First algorithm Exploration

Let's get this project starting!

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Import data
df_events = pd.read_csv('../data/input_data/events.csv')
df_halls = pd.read_csv('../data/input_data/halls.csv')
df_parking_lots_capacity = pd.read_csv('../data/input_data/parking_lots_capacity.csv')
df_parking_lots_distances = pd.read_csv('../data/input_data/parking_lots_distances.csv')

In [3]:
# In this version each event has one hall
df_events.head()

,event,date,status,demand,hall
0,inhorgenta,17.02.25,aufbau,280,A1
1,inhorgenta,18.02.25,aufbau,330,A1
2,inhorgenta,19.02.25,aufbau,420,A1
3,inhorgenta,20.02.25,aufbau,420,A1
4,inhorgenta,21.02.25,laufzeit,2278,A1


In [4]:
# Each hall has a distance to each entrance (with aritficial numbers for now)
df_halls.head()

,hall,distance_north,distance_north_east,distance_east,distance_west,distance_north_west
0,A1,200,300,160,60,100
1,A2,175,260,140,80,140
2,A3,150,220,120,100,180
3,A4,150,180,100,120,220
4,A5,175,140,80,140,260


In [5]:
# Each parking lot has a capacity to a given time (may change due to construnctions, etc)
df_parking_lots_capacity.head()

,parking_lot,capacity,valid_from,valid_to
0,P1 Nord (Tor 17a - Tor 11c),2750,01.01.25,02.03.25
1,P1 Nord (westl. Tor 17a),350,01.01.25,02.03.25
2,P2 Nord (östl. Tor 11c),500,01.01.25,02.03.25
3,P3,200,01.01.25,02.03.25
4,P4,100,01.01.25,02.03.25


In [6]:
# Each parking lot also has a distance to each entrance (with aritficial numbers for now)
df_parking_lots_distances.head()

,parking_lot,distance_north,distance_north_east,distance_east,distance_west,distance_north_west
0,P1 Nord (Tor 17a - Tor 11c),400,430,600,1000,900
1,P1 Nord (westl. Tor 17a),380,410,580,950,850
2,P2 Nord (östl. Tor 11c),420,390,500,1300,1200
3,P3,700,720,750,1300,1200
4,P4,800,830,850,1400,1300


In [7]:
# Merge events with halls to figure out distances of the events to the entrances
df_events_halls = df_events.merge(df_halls, on='hall')
df_events_halls.head()

,event,date,status,demand,hall,distance_north,distance_north_east,distance_east,distance_west,distance_north_west
0,inhorgenta,17.02.25,aufbau,280,A1,200,300,160,60,100
1,inhorgenta,18.02.25,aufbau,330,A1,200,300,160,60,100
2,inhorgenta,19.02.25,aufbau,420,A1,200,300,160,60,100
3,inhorgenta,20.02.25,aufbau,420,A1,200,300,160,60,100
4,inhorgenta,21.02.25,laufzeit,2278,A1,200,300,160,60,100


In [8]:
# Create a list to store the distances between halls and all parking lots
halls_parking_distances = []

# Iterate over each row in df_events_halls
for _, hall in df_halls.iterrows():
    # Iterate over each row in df_parking_lots_distances
    for _, parking_lot in df_parking_lots_distances.iterrows():
        # Calculate total distances
        distance_north = hall['distance_north'] + parking_lot['distance_north']
        distance_north_east = hall['distance_north_east'] + parking_lot['distance_north_east']
        distance_east = hall['distance_east'] + parking_lot['distance_east']
        distance_west = hall['distance_west'] + parking_lot['distance_west']
        distance_north_west = hall['distance_north_west'] + parking_lot['distance_north_west']
        
        # Append new row to distances list
        halls_parking_distances.append({
            'hall': hall['hall'],
            'parking_lot': parking_lot['parking_lot'],
            'distance_north': distance_north,
            'distance_north_east': distance_north_east,
            'distance_east': distance_east,
            'distance_west': distance_west,
            'distance_north_west': distance_north_west
        })

# Create df_halls_parking_distances DataFrame
df_halls_parking_distances = pd.DataFrame(halls_parking_distances)

In [9]:
# Now each hall has a distance to each parking lot via each entrance
df_halls_parking_distances.head()

,hall,parking_lot,distance_north,distance_north_east,distance_east,distance_west,distance_north_west
0,A1,P1 Nord (Tor 17a - Tor 11c),600,730,760,1060,1000
1,A1,P1 Nord (westl. Tor 17a),580,710,740,1010,950
2,A1,P2 Nord (östl. Tor 11c),620,690,660,1360,1300
3,A1,P3,900,1020,910,1360,1300
4,A1,P4,1000,1130,1010,1460,1400


In [10]:
# Merge events with halls and parking lots distances
df_events_halls_parking_distances = df_events.merge(df_halls_parking_distances, on='hall')

In [11]:
# Now each event has a distance to each parking lot via each entrance
# One row is an event at a specific date, it's hall and a specific parking lot with all distances (not to the entrance but to the parking lot)
df_events_halls_parking_distances.head()

,event,date,status,demand,hall,parking_lot,distance_north,distance_north_east,distance_east,distance_west,distance_north_west
0,inhorgenta,17.02.25,aufbau,280,A1,P1 Nord (Tor 17a - Tor 11c),600,730,760,1060,1000
1,inhorgenta,17.02.25,aufbau,280,A1,P1 Nord (westl. Tor 17a),580,710,740,1010,950
2,inhorgenta,17.02.25,aufbau,280,A1,P2 Nord (östl. Tor 11c),620,690,660,1360,1300
3,inhorgenta,17.02.25,aufbau,280,A1,P3,900,1020,910,1360,1300
4,inhorgenta,17.02.25,aufbau,280,A1,P4,1000,1130,1010,1460,1400


In [12]:
# Find the parking lot with the smallest distance for each event
# Merge the two dataframes
df_events_closest_parking_lot = pd.merge(df_events_halls_parking_distances, df_parking_lots_capacity, on='parking_lot')

# Filter rows where capacity >= demand
df_events_closest_parking_lot = df_events_closest_parking_lot[df_events_closest_parking_lot['capacity'] >= df_events_closest_parking_lot['demand']]

# Add a column for the minimum distance
df_events_closest_parking_lot['min_distance'] = df_events_closest_parking_lot[['distance_north', 'distance_north_east', 'distance_east', 'distance_west', 'distance_north_west']].min(axis=1)

# Sort by event and minimum distance, then drop duplicates, keeping the first (i.e., the one with the smallest distance)
df_events_closest_parking_lot = df_events_closest_parking_lot.sort_values(['event', 'min_distance']).drop_duplicates('event', keep='first')

In [13]:
# Now each event has a parking lot with the smallest distance to it
df_events_closest_parking_lot.head()

,event,date,status,demand,hall,parking_lot,distance_north,distance_north_east,distance_east,distance_west,distance_north_west,capacity,valid_from,valid_to,min_distance
996,Lopec,23.02.25,aufbau,140,C6,P2 Nord (östl. Tor 11c),520,440,560,1460,1400,500,01.01.25,02.03.25,440
333,Münchner Autotage,16.02.25,aufbau,200,B3,P1 Nord (westl. Tor 17a),480,550,690,1040,1030,350,01.01.25,02.03.25,480
633,fre.e + IMOT,14.02.25,aufbau,210,B4,P1 Nord (westl. Tor 17a),480,590,670,1060,990,350,01.01.25,02.03.25,480
3,inhorgenta,17.02.25,aufbau,280,A1,P1 Nord (westl. Tor 17a),580,710,740,1010,950,350,01.01.25,02.03.25,580
